In [9]:
import sys
from functools import partial
sys.path.append("../")
import genjax
import numpy as np
import bayes3d as b
import jax.numpy as jnp
import bayes3d.genjax
import jax
from utils import *
from viz import *
from models import *
from renderer_setup import *
from inference import *
from genjax.inference.importance_sampling import sampling_importance_resampling
from genjax._src.core.transforms.incremental import NoChange
from genjax._src.core.transforms.incremental import UnknownChange
from genjax._src.core.transforms.incremental import Diff

console = genjax.pretty()

In [ ]:
gt_path = "../ground_truths/genjax_generated/scene_6.pkl"
metadata = load_metadata(gt_path)
gt_images = metadata["rendered"]
model = eval("model_v{}".format(metadata["model_version"]))
RENDERER_ARGS = metadata["RENDERER_ARGS"]
setup_renderer_and_meshes = eval("setup_renderer_and_meshes_v{}".format(metadata["renderer_setup_version"]))
setup_renderer_and_meshes(**RENDERER_ARGS)

In [ ]:
def c2f_pose_update_v2(trace_, key, pose_grid, enumerator):
    
    scores = enumerator.enumerate_choices_get_scores(trace_, key, pose_grid)
    return enumerator.update_choices(
        trace_, key,
        pose_grid[scores.argmax()]
    )
c2f_pose_update_v2_jit = jax.jit(c2f_pose_update_v2, static_argnames=("enumerator",))



In [ ]:
trs, rendered = inference_approach_F(model, gt_images, metadata)

In [ ]:
print(np.sum(metadata["scores"]))
print(np.sum([tr.get_score() for tr in trs]))

In [ ]:
# video_from_trace(tr, framerate=5, use_retval=True)
# video_from_rendered(gt_images, framerate=5,)
# video_comparison_from_trace(tr,framerate = 5, scale = 4)
video_comparison_from_images(rendered, gt_images,framerate = 5, scale = 4)